### Assignment 3 - Algorithm Analysis (Due Mar 21)

---


## Completion requirements

Due: Friday, 21 March 2025, 11:59 PM


## Min/Max Search Limit Function 
Extend the minimum two value/indices function from the lecture to give more values and the choice of min/max. i.e. write a function in python that accepts 3 parameters

    - A list to search from
    - Choice of min/max
    - Number of elements
    - Dutch Flag Problem  

Extend the Dutch Flag Problem from the lecture to 3 colours. i.e. given a list of strings, each of which is either 'red', 'white', or 'blue' (each is repeated several times in the list), rearrange the list so that the strings are in the order of the Dutch national flag—all the 'red' strings first, then all the 'white' strings, then all the 'blue' strings. Write a function called dutch_flag that takes a list and solves this problem.
Generate random lists for n = 10, 100, 1000, 10000, 100000, 1000000 and test your function with these lists. Plot a graph of running times using matplotlib. Your graph should be a line. i.e. the Dutch Flag algorithm shown has O(n) running time
Compare Bubble sort with other sorting methods

Implement a function for Bubble sort similar to the other sort functions we discussed in class
Use the lecture notes as a starting point, create a Jupyter notebook that compares the running times of all sorting methods (insertion, selection, merge, bubble, pysort) by plotting the running times on the same graph

In [ ]:
import random

def dutch_flag(three_colour_flag: list, colours: list) -> list:

    flag_components: dict = {colours[i]: three_colour_flag.count(colours[i]) for i in range(len(colours))}

    print("Flag Components dictionary:")
    print(flag_components)
    print("\n")

    sorted_flag: list = []

    for color in colours:
        sorted_flag += [color] * flag_components[color]

    return sorted_flag

def main() -> None:
    colours: list = ['red', 'white', 'blue']
    three_colour_flag: list = random.choices(colours, k=12)

    print("OG Flag list:")
    print(three_colour_flag)
    print("\n")

    print(dutch_flag(three_colour_flag, colours))

if __name__ == "__main__":
    main()

OG Flag list:
['red', 'red', 'white', 'red', 'red', 'red', 'blue', 'blue', 'white', 'blue', 'red', 'blue']




NameError: name 'colors' is not defined